# Part I — Foundations  
## 4. Installing Django + First Project (The Right Way)

This chapter takes you from “I have a Python environment” to “I have a clean Django
project that:
- runs locally,
- has a professional structure,
- includes a first real endpoint,
- includes a health check,
- and is ready to grow without becoming messy.”

You’ll also connect your tooling from Chapter 3 (Ruff/Black/pre-commit/pytest) to a
real Django codebase.

---

## 4.0 Learning Outcomes (What you must be able to do)

By the end of this chapter, you should be able to:

1. Install Django in an isolated environment and pin dependencies.
2. Create a Django **project** and explain what each generated file does.
3. Create a Django **app** and explain the “project vs app” boundary.
4. Run the development server and understand what it is (and what it is not).
5. Add a simple page endpoint and a JSON health endpoint.
6. Add basic tests for those endpoints.
7. Understand the minimum settings that matter at the start:
   - `INSTALLED_APPS`, `MIDDLEWARE`, `TEMPLATES`, `DATABASES`, `STATIC_URL`,
     `ALLOWED_HOSTS`, `DEBUG`, `SECRET_KEY`
8. Avoid the most common early mistakes (import paths, wrong working directory,
   forgetting to add an app, etc.).

---

## 4.1 Install Django (Correctly) + Pin Dependencies

### 4.1.1 Why “installing Django” is more than `pip install django`
In professional projects, you want:
- reproducible builds (same versions everywhere)
- minimal “random upgrade broke everything” surprises
- clean separation of runtime vs dev dependencies

You already set up a venv and dev tooling. Now we add Django as a runtime dependency.

### 4.1.2 Decide where dependencies live
A common practical split:

- `requirements.txt` → runtime dependencies (Django + packages used by the app)
- `requirements-dev.txt` → dev-only tools (ruff, black, pytest, pre-commit)

If you currently have only dev tools in `requirements-dev.txt`, keep it that way.

### 4.1.3 Install Django
In your activated venv:

```bash
python -m pip install django
```

Verify:

```bash
python -c "import django; print(django.get_version())"
```

Now freeze runtime dependencies into `requirements.txt`:

```bash
python -m pip freeze > requirements.txt
```

**Why freeze now?**  
Because later you’ll deploy, run CI, or share the project—pinning ensures everyone
installs the same versions.

> Note: In more advanced setups you may use a lockfile tool (Poetry, pip-tools,
> uv, etc.). We’ll keep it simple and industry-acceptable with `requirements.txt`.

---

## 4.2 Create the Django Project (And Understand the Command)

### 4.2.1 What “project” means in Django
A Django **project** is the whole site/application configuration:
- settings (database, middleware, installed apps)
- top-level URL routing
- WSGI/ASGI entry points

A project can contain many **apps**.

### 4.2.2 Start the project
From your repo root (the directory where your `pyproject.toml` lives), run:

```bash
django-admin startproject config .
```

#### Explain every part of this command

- `django-admin` is Django’s command-line tool.
- `startproject` generates a new Django project.
- `config` is the **project package name** (you’ll import from it).
  - Many teams use `config`, `core`, or `project`.
  - Avoid naming it `django` or anything that conflicts with packages.
- `.` (dot) means: “create the project in the current directory.”

Why use `.`?
- Without it, Django creates a nested directory (often annoying):

If you run:
```bash
django-admin startproject config
```

You usually get:
```text
config/
  manage.py
  config/
    settings.py
```

With the dot version:
```text
manage.py
config/
  settings.py
```

This is cleaner at the repo root (common in real projects).

---

## 4.3 Understand the Generated Files (No Mystery Allowed)

After running the command, you’ll see something like:

```text
manage.py
config/
  __init__.py
  settings.py
  urls.py
  asgi.py
  wsgi.py
```

### 4.3.1 `manage.py` (your local command launcher)
`manage.py` is a convenience wrapper that:
- sets `DJANGO_SETTINGS_MODULE` to your settings
- lets you run Django commands reliably from the project context

You will use it constantly:

```bash
python manage.py runserver
python manage.py startapp pages
python manage.py makemigrations
python manage.py migrate
python manage.py test
```

**Why not use `django-admin` for everything?**  
You can, but `manage.py` is safer because it automatically points to your project’s
settings.

### 4.3.2 `config/settings.py` (the central configuration)
This file configures:
- installed apps
- middleware
- templates
- database
- static files
- security-related settings

This is where a lot of “production readiness” lives later.

### 4.3.3 `config/urls.py` (top-level routing)
This is the first routing table Django checks.

You will later:
- include app URL files here (`include(...)`)
- add admin routes
- add API routes

### 4.3.4 `config/wsgi.py` (production entry point for WSGI servers)
WSGI is the traditional synchronous interface used by:
- Gunicorn (WSGI mode)

You’ll care when deploying classic Django apps.

### 4.3.5 `config/asgi.py` (production entry point for ASGI servers)
ASGI supports async and protocols beyond HTTP (like WebSockets via Channels).

Even if you don’t use async now, ASGI is increasingly standard.

### 4.3.6 `config/__init__.py`
Marks `config/` as a Python package. Mostly empty.

---

## 4.4 Run the Development Server (And Understand What It Is)

Run:

```bash
python manage.py runserver
```

You’ll see output like:

- server running on `http://127.0.0.1:8000/`
- warnings if migrations are pending

Visit the URL in your browser.

### 4.4.1 What the dev server is
- A local development convenience server
- Auto-reloads code on change
- Shows detailed debug pages when `DEBUG=True`

### 4.4.2 What the dev server is NOT
- Not a production server
- Not optimized for performance/security
- Not intended to face the public internet

Industry rule:
- dev server for development only
- production uses Gunicorn/Uvicorn + Nginx/load balancer (later chapters)

---

## 4.5 First Database Setup: Migrate Immediately (Even If You Haven’t Made Models)

Django ships with built-in apps like auth and admin. They require tables.

Run:

```bash
python manage.py migrate
```

### Why this matters
Even before you create your own models, Django’s built-in components need:
- users table
- sessions table (if using DB sessions)
- admin-related tables

If you don’t migrate, you’ll hit runtime errors later (especially when logging in or
using admin).

---

## 4.6 Create Your First App (Project vs App in Real Terms)

### 4.6.1 What “app” means in Django
A Django **app** is a modular unit that provides a feature set, such as:
- blog
- accounts
- billing
- inventory
- pages
- API

Apps should be reusable and conceptually cohesive.

A **project** is the container that wires apps together into one site.

### 4.6.2 Create an app named `pages`
Run:

```bash
python manage.py startapp pages
```

You’ll get:

```text
pages/
  __init__.py
  admin.py
  apps.py
  migrations/
    __init__.py
  models.py
  tests.py
  views.py
```

### 4.6.3 Explain the generated app files
- `apps.py`: app configuration (name, default auto field, etc.)
- `views.py`: request handlers (you’ll write these)
- `models.py`: database models (later)
- `admin.py`: admin registrations (later)
- `tests.py`: tests (we’ll start now)
- `migrations/`: migration history for models

### 4.6.4 Add the app to `INSTALLED_APPS`
Open `config/settings.py`, find:

```python
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
]
```

Add your app:

```python
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "pages",
]
```

#### Why this step is required
If an app isn’t in `INSTALLED_APPS`, Django will not fully integrate it:
- templates discovery patterns may differ (later)
- model migrations won’t run
- signals and checks might not load
- some features won’t behave as expected

This is one of the most common beginner mistakes.

---

## 4.7 Create Your First View (HTML) and Wire a URL

### 4.7.1 What a “view” really is
A Django view is simply:
- a function (or class) that accepts an HTTP request
- returns an HTTP response

“View” does **not** mean HTML template only. It can return:
- HTML
- JSON
- redirects
- files

### 4.7.2 Create a simple page view
Edit `pages/views.py`:

```python
from django.http import HttpResponse


def home(request):
    return HttpResponse("Hello from Django pages.home")
```

#### Explain what’s happening
- `request` is a Django `HttpRequest` object representing the incoming HTTP request:
  it contains method, headers, user, GET params, POST data, etc.
- `HttpResponse(...)` creates an HTTP response with:
  - status code 200 by default
  - body equal to the string you pass
  - default content type `text/html; charset=utf-8` unless specified otherwise

### 4.7.3 Create a URLconf for the `pages` app
Create `pages/urls.py`:

```python
from django.urls import path

from . import views

urlpatterns = [
    path("", views.home, name="home"),
]
```

#### Explain `path("", ...)`
- The empty string `""` means “the root of this app’s URL space.”
- If later we include these URLs at the site root, `""` corresponds to `/`.

#### Explain `name="home"`
Naming URLs is an industry standard Django practice:
- templates can refer to routes by name rather than hardcoding paths
- refactoring URLs becomes safer

You’ll later use:

```python
from django.urls import reverse
reverse("home")
```

or in templates:

```django
{% url "home" %}
```

### 4.7.4 Include app URLs in the project URLconf
Edit `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
]
```

#### Explain `include("pages.urls")`
- `include(...)` imports another URL list and inserts it here.
- This keeps your project modular:
  - project-level `config/urls.py` remains small
  - each app manages its own routes

### 4.7.5 Run and verify
Run:

```bash
python manage.py runserver
```

Visit:

- `http://127.0.0.1:8000/` → should display your message
- `http://127.0.0.1:8000/admin/` → admin login page (works after migrations)

---

## 4.8 Add a Health Check Endpoint (Industry Standard)

A health endpoint is used by:
- load balancers
- Kubernetes readiness/liveness probes
- uptime monitors

### 4.8.1 Why `/healthz` is a thing (and why it’s usually simple)
Health checks should be:
- fast (milliseconds)
- reliable
- not dependent on heavy components unless you want a deeper check
- not requiring authentication

Common patterns:
- `/healthz` or `/health` returns “ok”
- sometimes also provide `/readyz` which checks DB/redis if needed

We’ll implement a minimal one now.

### 4.8.2 Create a JSON health view
Edit `pages/views.py`:

```python
from django.http import HttpResponse, JsonResponse


def home(request):
    return HttpResponse("Hello from Django pages.home")


def healthz(request):
    return JsonResponse(
        {
            "status": "ok",
        }
    )
```

#### Explain `JsonResponse`
`JsonResponse`:
- serializes a Python dict to JSON
- sets `Content-Type: application/json`
- ensures valid JSON output

This is safer than manually doing:

```python
HttpResponse(json.dumps(data), content_type="application/json")
```

because it standardizes encoding and defaults.

### 4.8.3 Wire it to URLs
Edit `pages/urls.py`:

```python
from django.urls import path

from . import views

urlpatterns = [
    path("", views.home, name="home"),
    path("healthz/", views.healthz, name="healthz"),
]
```

Now:
- `GET /healthz/` returns JSON

Test in browser or with curl:

```bash
curl -i http://127.0.0.1:8000/healthz/
```

You should see:
- status 200
- `Content-Type: application/json`
- body like `{"status":"ok"}`

---

## 4.9 Introduce Query Strings in a Real Endpoint (Because You’ll Use Them Everywhere)

Let’s add a toy endpoint to demonstrate query parameters (ties back to Chapter 1).

### 4.9.1 Add an “echo” endpoint
Edit `pages/views.py`:

```python
from django.http import HttpResponse, JsonResponse


def home(request):
    return HttpResponse("Hello from Django pages.home")


def healthz(request):
    return JsonResponse({"status": "ok"})


def echo(request):
    message = request.GET.get("message", "")
    return JsonResponse(
        {
            "message": message,
            "method": request.method,
        }
    )
```

#### Explain `request.GET`
- `request.GET` is a mapping of query string parameters.
- For a URL like:
  - `/echo/?message=hello`
- `request.GET.get("message")` returns `"hello"`

Important:
- Query parameters are strings.
- Even if you send `?page=2`, you get `"2"` and must parse to `int` yourself.

### 4.9.2 Add the URL
Edit `pages/urls.py`:

```python
from django.urls import path

from . import views

urlpatterns = [
    path("", views.home, name="home"),
    path("healthz/", views.healthz, name="healthz"),
    path("echo/", views.echo, name="echo"),
]
```

Test:

```bash
curl -i "http://127.0.0.1:8000/echo/?message=Hello%20Django"
```

---

## 4.10 Django Settings: The Minimum You Must Understand Right Now

Open `config/settings.py`. We won’t rewrite everything yet, but you must understand
these sections early.

### 4.10.1 `DEBUG`
- `DEBUG=True` gives helpful error pages and auto-reload.
- `DEBUG=False` is production mode: safer, less info leakage, different behavior.

Industry rule:
- Never deploy with `DEBUG=True`.

### 4.10.2 `SECRET_KEY`
This key is used for cryptographic signing (sessions, password reset tokens, etc.).

If an attacker gets your `SECRET_KEY`, they can potentially:
- forge signed cookies/session data (depending on setup)
- compromise security mechanisms

Industry rule:
- keep it secret (env var in production)
- rotate carefully (advanced topic later)

### 4.10.3 `ALLOWED_HOSTS`
This prevents Host header attacks.

In production you set it to your real domains, e.g.:

```python
ALLOWED_HOSTS = ["example.com", "www.example.com"]
```

In dev, Django often defaults to `[]` and is permissive with debug.

### 4.10.4 `INSTALLED_APPS`
Controls what Django loads. You added `"pages"` here.

### 4.10.5 `MIDDLEWARE`
Middleware wraps every request/response.
Examples:
- security middleware
- session middleware
- CSRF middleware
- authentication middleware

You’ll study middleware deeply later; for now, know it’s part of the request pipeline.

### 4.10.6 `TEMPLATES`
Controls template engine configuration. We’ll use templates in the next chapter,
but you should recognize:
- template directories
- app template discovery
- context processors

### 4.10.7 `DATABASES`
Default is SQLite in dev (easy, file-based). That’s fine for learning.

Industry reality:
- production usually uses PostgreSQL
- but learning Django is easiest with SQLite first

---

## 4.11 Connect Tooling to Django (Ruff/Black/Pre-commit + Django Files)

### 4.11.1 Ensure Ruff/Black run cleanly on Django code
Run:

```bash
python -m ruff check .
python -m black . --check
python -m pytest
```

At this moment, pytest may find no tests or pass a trivial test.

### 4.11.2 Common Ruff noise in Django projects (and how teams handle it)
Django generates files like migrations that you typically don’t want to lint too
strictly early on.

A common approach is to exclude migrations from linting.

In `pyproject.toml` (Ruff section), you can add:

```toml
[tool.ruff]
line-length = 88
target-version = "py311"
extend-exclude = ["*/migrations/*"]
```

**Why exclude migrations?**
- migrations are auto-generated
- linting them can create busywork with little value

Industry standard: exclude or keep lint rules minimal for migrations.

---

## 4.12 Add Real Tests for Your First Endpoints

Django has a built-in test framework (unittest-based). You can later switch to
pytest-django if you prefer; many teams do. For now, we’ll use Django’s default,
because it requires zero extra packages and teaches fundamentals.

### 4.12.1 Write tests in `pages/tests.py`
Edit `pages/tests.py`:

```python
from django.test import Client, TestCase


class PagesTests(TestCase):
    def setUp(self):
        self.client = Client()

    def test_home_returns_200(self):
        response = self.client.get("/")
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "Hello from Django")

    def test_healthz_returns_json_ok(self):
        response = self.client.get("/healthz/")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response["Content-Type"], "application/json")
        self.assertJSONEqual(response.content, {"status": "ok"})

    def test_echo_reads_query_param(self):
        response = self.client.get("/echo/?message=hello")
        self.assertEqual(response.status_code, 200)
        self.assertJSONEqual(
            response.content,
            {
                "message": "hello",
                "method": "GET",
            },
        )

    def test_echo_defaults_message_to_empty_string(self):
        response = self.client.get("/echo/")
        self.assertEqual(response.status_code, 200)
        self.assertJSONEqual(
            response.content,
            {
                "message": "",
                "method": "GET",
            },
        )
```

#### Explain what this test code is doing (line-by-line concepts)

- `TestCase` sets up an isolated test environment:
  - uses a test database (even if your tests don’t touch DB, Django prepares it)
  - ensures tests are repeatable
- `Client()` is Django’s test HTTP client:
  - it simulates making requests to your Django app without running a real server
  - `self.client.get("/healthz/")` calls routing + views internally
- `assertContains` checks:
  - status code is 200 by default
  - response body contains the expected text
- `assertJSONEqual` parses JSON and compares structures (better than string compare)

### 4.12.2 Run tests
```bash
python manage.py test
```

Why `python manage.py test` (not `pytest`) right now?
- this uses Django’s built-in runner
- guaranteed to work immediately without extra setup
- later we’ll integrate pytest-django (common in industry)

---

## 4.13 Common Beginner Errors (And Exactly How to Fix Them)

### Error A: “ModuleNotFoundError: No module named 'pages'”
Cause:
- you created the app but didn’t add it to `INSTALLED_APPS`, or
- you’re running commands from the wrong directory / wrong Python environment

Fix checklist:
1. Ensure venv is active.
2. Ensure `"pages"` is in `INSTALLED_APPS`.
3. Run commands from the directory that contains `manage.py`.

### Error B: “You have unapplied migrations”
Cause:
- you didn’t run `migrate` after creating the project

Fix:
```bash
python manage.py migrate
```

### Error C: “404 Not Found” for `/`
Cause:
- you didn’t include `pages.urls` in `config/urls.py`
- or you wrote incorrect URL patterns

Fix:
- verify `path("", include("pages.urls"))` exists
- verify `pages/urls.py` defines `path("", views.home, ...)`

### Error D: Tests fail because of content type mismatch
JSON responses often include charset. If your Django version returns something like:
`application/json` vs `application/json; charset=utf-8`, don’t overfit the test.

Safer test pattern:
- check it starts with application/json

Example adjustment:

```python
self.assertTrue(response["Content-Type"].startswith("application/json"))
```

(Use this if needed.)

---

## 4.14 Exercises (Do these before moving on)

1. Add an `/about/` page:
   - route: `/about/`
   - returns HTML via `HttpResponse`
   - write a test that checks it contains your name or project name

2. Add a `/time/` endpoint that returns JSON:
   - keys: `{"server_time": "...", "timezone": "..."}`
   - keep it simple (string is fine)
   - write tests that verify keys exist (don’t assert an exact time string)

3. Add a query parameter behavior:
   - `/echo/?uppercase=true&message=hi`
   - if `uppercase=true`, return `"HI"`
   - write tests for both paths

4. Make sure these commands all succeed:
```bash
python -m ruff check .
python -m black . --check
python manage.py test
```

---

## 4.15 Chapter Summary (What you now have)

You now have:
- a real Django project (`config/`)
- a real Django app (`pages/`)
- working routes for:
  - `/` (HTML)
  - `/healthz/` (JSON)
  - `/echo/` (query strings)
- basic test coverage proving endpoints behave correctly
- tooling that can keep the project clean as it grows

---

Next chapter is **Part I — Chapter 5: Django Architecture Overview** (MTV, request
flow, middleware, settings, WSGI/ASGI, static/media, and environments) — where we
turn what you just built into a clear mental model so you can scale from “hello
world” to real applications without confusion.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='3. developer_tooling_and_workflow.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='5. django_architecture_overview.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
